# Module 1. EDA and Data Split
---

본 모듈에서는 EDA(Exploratory Data Analysis; 탐색적 데이터 분석)을 통해 데이터를 간단하게 살펴 보겠습니다.

In [1]:
%load_ext autoreload
%autoreload 2
!pip -q install --upgrade pip nvidia-ml-py3
!pip -q install --upgrade awscli boto3 pandas joblib pyarrow iterative-stratification
!pip -q install --upgrade kaggle sagemaker

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob2
import joblib

import pyarrow
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont

WIDTH = 236
HEIGHT = 137
data_dir = './input'
raw_data_dir = './input/raw'

<br>

## 1. Prepraing Dataset
---
### Download dataset


#### Option 1. Download using Kaggle API
Kaggle API를 통해 데이터셋을 다운로드받은 후, 압축을 해제합니다. 코드 셀 수행을 위해서 Kaggle에 가입하여 `kaggle.json`을 다운로드받으셔야 합니다.

[주의] 아래 작업을 로컬에서 원활히 수행하려면 16GB 이상의 메모리가 필요합니다.

In [ ]:
# %%bash -s {raw_data_dir}
# rm -rf ~/.kaggle
# mkdir ~/.kaggle
# chmod 600 kaggle.json
# cp kaggle.json ~/.kaggle/kaggle.json
# kaggle competitions download -c bengaliai-cv19 -p $1

In [ ]:
# !unzip {raw_data_dir}/bengaliai-cv19.zip -d {data_dir}

In [ ]:
# raw_files_train = sorted(glob2.glob(f'{data_dir}/train_*.parquet')); print(raw_files_train)
# raw_files_test = sorted(glob2.glob(f'{data_dir}/test_*.parquet')); print(raw_files_test)

In [ ]:
# for file in raw_files_train:
#     newfile = file.replace('parquet', 'feather')
#     print(f'Converting parquet to feather - {file}')
#     data = pd.read_parquet(file)
#     data.to_feather(newfile)
#     os.remove(file)

In [ ]:
# for file in raw_files_test:
#     newfile = file.replace('parquet', 'feather')
#     print(f'Converting parquet to feather - {file}')
#     data = pd.read_parquet(file)
#     data.to_feather(newfile)
#     os.remove(file)

#### Option 2. Download from Amazon S3
Amazon S3에 저장된 데이터셋을 다운로드합니다.

In [4]:
!rm -rf $data_dir
!aws s3 cp --recursive s3://daekeun-workshop-public-material/bangali-handwritten/inputs $data_dir

In [ ]:
files_train = sorted(glob2.glob(f'{data_dir}/train_*.feather')); print(files_train)
files_test = sorted(glob2.glob(f'{data_dir}/test_*.feather')); print(files_test)

### Read feather files

훈련 데이터셋은 모두 4개의 feather 파일들로 구성되어 있고, 각 feather 파일은 32,333개의 컬럼으로 구성된 5만여 장의 이미지 데이터가 포함되어 있습니다.
- $32,333 = 1+(137 \times 236)$, 첫번째 컬럼은 Train Index로 이를 통해 정답 레이블을 알 수 있습니다.
- $137$: Height, $236$: Width

참고로, Kaggle의 원 데이터는 parquet 파일이며 parquet 파일은 `to_feather()` 메서드로 feather 포맷으로 쉽게 변환할 수 있습니다.

In [ ]:
%%time
train_img0 = pd.read_feather(files_train[0])
print(train_img0.shape)

In [ ]:
train_img0.head()

<br>

## 2. EDA (Exploratory Data Analysis)
---

### Check raw images

실제 raw 이미지 데이터를 확인해 봅니다. 아래 셀을 여러 번 반복해서 실행해 보세요.

In [ ]:
num_train = len(train_img0)
idx = np.random.randint(num_train)
raw_img = train_img0.iloc[idx, 1:].values.astype(np.uint8)
plt.imshow(255 - raw_img.reshape(HEIGHT, WIDTH), cmap='gray')

In [ ]:
train_df = pd.read_csv('./input/train.csv')
display(train_df.head())
print(f'Number of unique grapheme roots: {train_df["grapheme_root"].nunique()}')
print(f'Number of unique vowel diacritic: {train_df["vowel_diacritic"].nunique()}')
print(f'Number of unique consonant diacritic: {train_df["consonant_diacritic"].nunique()}')
print(f'Number of training data: {train_df.shape}')

In [ ]:
class_map_df = pd.read_csv('./input/class_map.csv')
display(class_map_df.head())
print(f'Size of class map: {class_map_df.shape}')      

### Most used top 10 Grapheme Roots in training set

In [ ]:
def get_n(df, class_map_df, field, n, top=True):
    top_graphemes = df.groupby([field]).size().reset_index(name='counts')['counts'].sort_values(ascending=not top)[:n]
    top_grapheme_roots = top_graphemes.index
    top_grapheme_counts = top_graphemes.values
    top_graphemes = class_map_df[class_map_df['component_type'] == field].reset_index().iloc[top_grapheme_roots]
    top_graphemes.drop(['component_type', 'label'], axis=1, inplace=True)
    top_graphemes.loc[:, 'count'] = top_grapheme_counts
    return top_graphemes

def image_from_char(char, fontsize=120):
    image = Image.new('RGB', (WIDTH, HEIGHT))
    draw = ImageDraw.Draw(image)
    myfont = ImageFont.truetype('kalpurush-2.ttf', fontsize)
    w, h = draw.textsize(char, font=myfont)
    draw.text(((WIDTH - w) / 2,(HEIGHT - h) / 3), char, font=myfont)
    return image

In [ ]:
top_10_roots = get_n(train_df, class_map_df, 'grapheme_root', 10)
top_10_roots

In [ ]:
f, ax = plt.subplots(2, 5, figsize=(12, 5))
ax = ax.flatten()

for i in range(10):
    ax[i].imshow(image_from_char(top_10_roots['component'].iloc[i]), cmap='gray')

### Top 5 Vowel Diacritic in taining data

In [ ]:
top_5_vowels = get_n(train_df, class_map_df, 'vowel_diacritic', 5)
top_5_vowels

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(12, 3))
ax = ax.flatten()

for i in range(5):
    ax[i].imshow(image_from_char(top_5_vowels['component'].iloc[i]), cmap='gray')

### Top 5 Consonants Diacritic in taining data

In [ ]:
top_5_consonants = get_n(train_df, class_map_df, 'consonant_diacritic', 5)
top_5_consonants

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(12, 3))
ax = ax.flatten()

for i in range(5):
    ax[i].imshow(image_from_char(top_5_consonants['component'].iloc[i]), cmap='gray')

### Distribution of classes

In [ ]:
def plot_count(feature, title, df, size=1):
    '''
    Plot count of classes of selected feature; feature is a categorical value
    param: feature - the feature for which we present the distribution of classes
    param: title - title to show in the plot
    param: df - dataframe 
    param: size - size (from 1 to n), multiplied with 4 - size of plot
    '''
    f, ax = plt.subplots(1, 1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3, '{:1.2f}%'.format(100*height/total),ha="center") 
    plt.show() 

In [ ]:
plot_count('grapheme_root', 'grapheme_root (train)', train_df, size=3)

In [ ]:
plot_count('vowel_diacritic', 'vowel_diacritic (train)', train_df, size=3)

In [ ]:
plot_count('consonant_diacritic', 'consonant_diacritic (train)', train_df, size=2)

<br>

## 3. Split Train/Validation Data 

1개의 레이블이 아닌 multi-label에 대한 층화추출이 필요하면 아래 라이브러리를 설치하시면 됩니다.

In [ ]:
train_df['id'] = train_df['image_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
train_df.tail()

In [ ]:
cols = ['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
X = train_df[cols].values[:,0]
y = train_df[cols].values[:,1:]

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mask = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
train_df['fold'] = -1

for i, (trn_idx, vld_idx) in enumerate(mask.split(X,y)):
    train_df.loc[vld_idx, 'fold'] = i

In [ ]:
train_df['fold'].value_counts()

fold0 훈련 데이터셋의 클래스 분포가 전체 훈련 데이터의 분포와 동일한지 확인해 봅니다.

In [ ]:
train_df_fold0 = train_df[train_df['fold']==0]
plot_count('vowel_diacritic', 'vowel_diacritic (train)', train_df_fold0, size=3)

fold 컬럼이 저장된 데이터프레임을 별도의 csv파일로 저장합니다. k-fold cross validation 시에, 이 csv를 재사용할 수 있습니다. 

In [ ]:
train_df.to_csv(f'{data_dir}/train_folds.csv', index=False)

<br>

## 4. Copy Files to S3

아래 코드 셀은 로컬 환경에 저장되어 있는 훈련 데이터를 S3로 전송합니다. 네트워크 속도가 빠르지 않다면 수 분이 소요될 수 있습니다.
참고로, 기본 버킷은 `'sagemaker-[YOUR REGION]-[YOUT ACCOUNT ID]'` 으로 자동으로 지정되어 있고, 여러분의 고유한 S3 생성 후 이를 지정할 수도 있습니다.

In [ ]:
%%time
import boto3, os
import sagemaker
bucket = sagemaker.Session().default_bucket()
prefix = 'bangali/train'
s3_bucket = boto3.Session().resource('s3').Bucket(bucket)

for file in files_train:
    f = file.split('/')[-1]
    s3_bucket.Object(os.path.join(prefix, f)).upload_file(file)
    
s3_bucket.Object(os.path.join(prefix, 'train_folds.csv')).upload_file(f'{data_dir}/train_folds.csv')